# Strongly Connected Components

Groups in a directed graph are said to be [strongly connected](https://en.wikipedia.org/wiki/Strongly_connected_component) if there exists a path between each node in the group.

In the graph below, there are three strongly connected components: `a b e`, `f g`, and `c d h`.



<img width="100%" height="100%" src="images/scc_graph.png">

In [1]:
graph = {'a': ['b'],
         'b': ['c', 'e', 'f'],
         'c': ['d', 'g'],
         'd': ['c', 'h'],
         'e': ['a', 'f'],
         'f': ['g'],
         'g': ['f'],
         'h': ['d', 'g']}

## Kosaraju's algorithm

[Wikipedia](https://en.wikipedia.org/wiki/Kosaraju%27s_algorithm) | [NetworkX Implementation](https://networkx.org/documentation/stable/_modules/networkx/algorithms/components/strongly_connected.html#kosaraju_strongly_connected_components)

In [2]:
from collections import defaultdict


def components(graph):
    transposed = defaultdict(list)
    
    for u in graph:
        for v in graph[u]:
            transposed[v].append(u)

    visited = set()
    nodes = []
    components = defaultdict(set)
    
    def visit(u):
        if u not in visited:
            visited.add(u)
            
            for v in graph[u]:
                visit(v)

            nodes.insert(0, u)
                
    for u in graph:
        visit(u)
    
    def assign(u, root):
        if u in visited:
            visited.remove(u)
            
            components[root].add(u)

            for v in transposed[u]:
                assign(v, root)      
       
    for u in nodes:
        assign(u, u)
        
    return list(components.values())


components(graph)

[{'a', 'b', 'e'}, {'c', 'd', 'h'}, {'f', 'g'}]

In [3]:
import networkx as nx


def nx_components(graph):
    graph = nx.DiGraph(graph)

    return list(nx.kosaraju_strongly_connected_components(graph))


mine = components(graph)
reference = nx_components(graph)

assert len(mine) == len(reference)

for component in mine:
    assert component in reference

## Tarjan's strongly connected components algorithm

[Wikipedia](https://en.wikipedia.org/wiki/Tarjan%27s_strongly_connected_components_algorithm) | [NetworkX Implementation](https://networkx.org/documentation/stable/_modules/networkx/algorithms/components/strongly_connected.html#strongly_connected_components)

In [4]:
def components(graph):
    index = 0            
    indexes = {}
    low_links = {}
    stack = []
    components = []

    def dfs(u):
        nonlocal index
        
        indexes[u] = low_links[u] = index
        index += 1

        stack.append(u)
        
        for v in graph[u]:
            if v not in indexes:
                dfs(v)
                low_links[u] = min(low_links[u], low_links[v])
            elif v in stack:
                low_links[u] = min(low_links[u], indexes[v])
                
        if low_links[u] == indexes[u]:
            component = set()
            v = None

            while v != u:
                component.add(v := stack.pop())

            components.append(component)
            
    for u in graph:
        if u not in indexes:
            dfs(u)
            
    return components


components(graph)

[{'f', 'g'}, {'c', 'd', 'h'}, {'a', 'b', 'e'}]

In [5]:
import networkx as nx


def nx_components(graph):
    graph = nx.DiGraph(graph)
        
    return list(nx.strongly_connected_components(graph))


mine = components(graph)
reference = nx_components(graph)

assert len(mine) == len(reference)

for component in mine:
    assert component in reference